# Introduction: Modeling

According to the No Free Lunch Theorem, no algorithm will beat random guessing across all learning tasks. What this means in terms of machine learning is that there is no one superior algorithm that does better for every task. Therefore, to find the best model for a given task, we simply have to test out many and find which one achieves the highest generalization performance on the test set. Machine learning is a largely empirical area, and finding the best model remains a process of experimentation.

With that in mind, in this notebook we will implement a number of machine learning models for the building energy prediction task. This is a supervised regression problem where we are asked to build a model to predict energy consumption based on historical energy usage and weather conditions. We will develop the models as functions that we can then evaluate across an entire range of buildings in order to find the best-performing model. Then we can select the best model for further development in the form of hyperparameter tuning. 

## Metric: Mean Absolute Percentage Error

In order to compare models, we need a single metric that we can use to assess performance. There are many choices for regression, including the root mean squared error (RMSE). RMSE is measured in the same units as the target, but suffers from the issue that the value depends on the magnitude of the targets and therefore cannot be compared between buildings that have different energy use. To compare performance across buildings, we need a metric that normalizes the error by the magnitude of the target. The mean absolute percentage error (MAPE) is one such metric. This performance measure has been used in previous electricity prediction studies, and can be used to compare predictions between buildings because it measures the error in terms relative the magnitude of the energy usage. The MAPE is calculated as

$$\mbox{MAPE} = 100\% * \frac{1}{n}\sum_{i=1}^n  \left|\frac{y_i-\hat{y}_i}{y_i}\right|$$

where $n$ is the number of observations, $y_i$ is the actual prediction for observation $i$, and $\hat{y}_i$ is the predicted value. We will calculate both the MAPE on the training data and on the testing data in order to determine the degree of overfitting in the model. In addition to the MAPE, we can calculate the time it takes for the model to learn. Although time is not a significant consideration when massive computing power is available, it is still an interesting comparison to make. 

## Models to Evaluate

We will evaluate the following models in this notebook. 
 
1. Elastic Net Linear Regression with $l1_{ratio} = 0.5$
2. K-Nearest Neighbors Regression with k = 10
3. Support Vector Machine Regression with Radial Basis Function Kernel
4. Random Forest Regression with 1000 decision trees 
5. Extra Trees Regression with 1000 decision trees
6. AdaBoosting Regression with 1000 decision trees as the base learner

Two other models with be evaluated in additional notebooks:

1. Gradient Boosting Machine
2. Deep Fully Connected Neural Networks

We will implement each model here one at a time, and then refactor the code into functions that can be called for any building dataset. The end outcome is a series of functions, one for each model, that can take in training features, training labels, testing features, and testing labels and return the performance of the model. 

### Imports

We will use a standard stack of data science libraries: `pandas`, `numpy`, `sklearn`, `matplotlib`. See the `requirements.txt` file for the correct version of these libraries to install. 

In [103]:
# numpy and pandas for data manipulation
import pandas as pd
import numpy as np

# Sklearn preprocessing functionality
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

# Matplotlib for visualizations
import matplotlib.pyplot as plt

# Adjust default font size 
plt.rcParams['font.size'] = 18

## Read in Dataset and Apply Feature Preprocessing

Let's read in an example dataset and preprocess the features using the function we developed earlier. This function takes in a building energy data, a number of days to use for testing, and a boolean for whether or not to scale the features. We will use 183 days (6 months) for testing and will choose to scale the features because we are using models that depend on the distance between observations to make predictions. Feature scaling is a best practice when comparing multiple algorithms so that the range of the features does not impact model performance. 

In [104]:
# Import the feature preprocessing
from utilities import preprocess_data

df = pd.read_csv('../data/f-APS_weather.csv')

# Preprocess the data for machine learning
train, train_targets, test, test_targets = preprocess_data(df, test_days = 183, scale = True)

train.head()

,timestamp,biz_day,week_day_end,ghi,dif,gti,temp,rh,pwat,ws,...,yday_cos,month_sin,month_cos,wday_sin,wday_cos,num_time_sin,num_time_cos,sun_rise_set_neither,sun_rise_set_rise,sun_rise_set_set
0,0.000000,1.0,0.0,0.0,0.0,0.0,0.394397,0.245303,0.069231,0.295082,...,0.629749,0.066987,0.75,1.0,0.25,0.500000,1.000000,1.0,0.0,0.0
1,0.000011,1.0,0.0,0.0,0.0,0.0,0.390086,0.250522,0.067949,0.295082,...,0.629749,0.066987,0.75,1.0,0.25,0.533050,0.998907,1.0,0.0,0.0
2,0.000022,1.0,0.0,0.0,0.0,0.0,0.387931,0.256785,0.067949,0.295082,...,0.629749,0.066987,0.75,1.0,0.25,0.565955,0.995631,1.0,0.0,0.0
3,0.000034,1.0,0.0,0.0,0.0,0.0,0.383621,0.262004,0.067949,0.303279,...,0.629749,0.066987,0.75,1.0,0.25,0.598572,0.990187,1.0,0.0,0.0
4,0.000045,1.0,0.0,0.0,0.0,0.0,0.379310,0.268267,0.067949,0.303279,...,0.629749,0.066987,0.75,1.0,0.25,0.630758,0.982600,1.0,0.0,0.0


The data is ready for machine learning. All of the values are numeric, the features have been scaled to between 0 and 1, and there are no missing values. 

# Machine Learning

For each model, we will calculate three stats: the mean absolute percentage error, the time to train (`fit`), and the time to test (`predict`). MAPE is calculated from the definition of the metric using the predicted values and the known test targets. The training and testing times are determined using the `default_timer` class from the `timeit` module. The `default_timer` automatically adjusts for operating system and provides the best timer accordingly. We will measure the wall-clock time, which is the total time to execute a program. This differs from the CPU time (also called execution time) which measures the time a CPU spent executing a specific program. Since we are mostly interested in the time out of curiosity rather than for model selection, the minor distinctions in time are not a significant concern. 

All of the models in this notebook will be built using Scikit-Learn. This open-source Python library benefits from a consistent syntax for using models which makes it extremely simple to quickly implement many models. There are three steps to using a machine learning model in Scikit-Learn:

1. Instantiate the model specifying hyperparameters
2. `fit`: train the model on the training data
3. `predict`: make predictions on the testing data

These same three steps apply to every model in Scikit-Learn. 

## Model Hyperparameters

Model hyperparameters can best be thought of as settings for a machine learnig model. In contrast to model parameters, which are learned by the model during training, model hyperparameters are set by the developer before training. As an example, the number of trees in a random forest is a model hyperparameter while the weights learned during a linear regression are model parameters. The choice of hyperparameters can have a siginficant affect both on the model's training and on the test performance. Much like choosing the best model is an experimental process, optimizing the hyperparameters of a selected model requires evaluating many different combinations to find the one that performs the best. This can be an extremely time and computational resource-intensive process. 

For this notebook, we will stick with the Scikit-Learn hyperparameters for the most part except where noted. Scikit-Learn aims to provide a set of reasonable default hyperparameters designed to get practicioners up and running with a decent model quickly. However, these hyperparameters are likely to be nowhere near optimal, and tuning them is recommended after a working system is built. If we had unlimited time, we would try out not only multiple models, but also work on optimizing the hyperparameters of each model to the problem. For now, we will stick with the default hyperparameters for comparing models and then optimize the hyperparameters of the model which performs the best. We will note that this might not be the most fair comparison because of the model dependence on hyperparameters, but it is a limitation we have to work with. (Note that for models where it is applicable, we set `n_jobs=-1` to use all available cores on the machine.

In this notebook, we focus on implementing the algorithms rather than explaining how they work. Two extremely good resources for learning the theory of these models are An Introduction to Statistical Learning, and Hands-On Machine Learning in Python with Scikit-Learn and TensorFlow. Both of these books provide the theory as well as how to implement the methods in R and Python respectively. Machine learning is only mastered through doing, and both of these books place a large emphasis on practicing the techniques described. With those considerations in mind, let's get to modeling! 

For the first model, we will walk through the steps individually, and then we will refactor the steps into a function that we can build for each model. 

## ElasticNet Linear Regression

ElasticNet Linear Regression is a regularized version of Linear Regression. It is a mix of two popular regularization methods: Lasso Regression and Ridge Regression. The blend of these two methods is controlled by the `l1_ratio` hyperparameter with the overall amount of regularization controlled by the `alpha` hyperparameter. We will use the default values of these hyperparameters which are `l1_ratio = 0.5` and `alpha = 1.0`. Unlike ordinary least squares (OLS) linear regression, which has an analytical solution, ElasticNet must be solved by coordinate descent. The equation for the parameters $\beta$ in ElasticNet is:

$$\hat{\beta} = \underset{\beta}{\operatorname{argmin}} (\| y-X \beta \|^2 + \alpha \lambda_1 \|\beta\|_1 + \alpha (1 - \lambda_1) \|\beta\|_2^2)$$

where $\lambda_1$ multiplies the L1 norm of the parameters and $1 - \lambda_1$ multiplies the L2 norm of the parameters. If $\alpha = 0$, then ElasticNet simplifies to ordinary least squares regression with an objective function of Mean Squared Error (MSE). 

In [105]:
from sklearn.linear_model import ElasticNet

# Timing utilities
from timeit import default_timer as timer

In [106]:
# Set up the model with default hyperparameters
model = ElasticNet(alpha = 1.0, l1_ratio=0.5)
model

ElasticNet(alpha=1.0, copy_X=True, fit_intercept=True, l1_ratio=0.5,
      max_iter=1000, normalize=False, positive=False, precompute=False,
      random_state=None, selection='cyclic', tol=0.0001, warm_start=False)

### Training 

Training in Scikit-Learn is done using the `fit` method (function) of a model. This takes in the training features and the targets. 

In [107]:
# Start the timer
train_start = timer()

# Train the model
model.fit(train, train_targets)

# Stop the timer
train_end = timer()

train_time = train_end - train_start

### Testing 

Making predictions in Scikit-Learn using the `predict` method of a model. It takes in only the testing features.

In [108]:
# Start the time
test_start = timer()

# Make predictions on testing data
predictions = model.predict(test)

# Stop the time
test_end = timer()

test_time = test_end - test_start

# Calculate the mape
mape = 100 * np.mean( abs(predictions - test_targets) / test_targets)

In [109]:
print("Training time (seconds): %0.4f" % train_time)
print("Prediction time (seconds): %0.4f" % test_time)
print("Testing MAPE: %0.2f" % mape)

Training time (seconds): 0.0623
Prediction time (seconds): 0.0193
Testing MAPE: 56.97


The three results can be recorded in a numpy array along with the name of the model. When we get results from multiple models, we can stack the results and then record them in a dataframe. 

In [110]:
# Record the results in a numpy array
results = np.array(['ElasticNet', train_time, test_time, mape])

results

array(['ElasticNet', '0.06234172316362674', '0.019255785629866295',
       '56.973741837828975'], dtype='<U20')

## Modeling Function

Let's now take the individual steps and put them into a single function. The function will operate the exact same for each model allowing for an efficient standardized workflow. This function takes in a model, training features, training labels, testing features, and testing labels, and returns the 3 numeric results along with the name of the model in a numpy array. One model will be able to handle any Scikit-Learn supervised regression model. 

In [111]:
def implement_model(model, train, training_targets, test, testing_targets, model_name):
    """Train a machine learning model and make predictions on a test set
    
    Parameters
    --------
    model : Scikit-Learn model object
        Model to use for training and making predictions
    
    train : dataframe, shape = [n_training_samples, n_features]
        Set of training features for training a model
    
    train_targets : array, shape = [n_training_samples]
        Array of training targets for training a model
        
    test : dataframe, shape = [n_testing_samples, n_features]
        Set of testing features for making predictions with a model
    
    test_targets : array, shape = [n_testing_samples]
        Array of testing targets for evaluating the model predictions
        
    model_name : string
        Name of the model used for returning results
        
    Returns
    --------
    
    results : array, shape = [4]
        Numpy array of results. 
        First index is the model, second is the training time,
        third is the testing time, and fourth is the MAPE. All indexes
        are in strings and so will need to be converted to numbers.
    
    """
    
    # Preprocess the data for machine learning
    train, train_targets, test, test_targets = preprocess_data(df, test_days = 183, scale = True)
    
    train_start = timer()
    
    # Start the timer
    train_start = timer()

    # Train the model
    model.fit(train, train_targets)

    # Calculate training time
    train_end = timer()
    train_time = train_end - train_start

    # Start test timer
    test_start = timer()

    # Make predictions
    predictions = model.predict(test)

    # Calculate testing time
    test_end = timer()
    test_time = test_end - test_start

    # Calculate the mape
    mape = 100 * np.mean( abs(predictions - test_targets) / test_targets)

    # Record the results
    results = [model_name, train_time, test_time, mape]
    
    return results


In [114]:
elasticnet_results = implement_model(model, train, train_targets, test, 
                                     test_targets, model_name = 'elasticnet')
elasticnet_results

['ElasticNet', 0.05524775643607427, 0.0011379254610801581, 56.973741837828975]

Now we can use this function for any Scikit-Learn machine learning model. We will go through the rest of the models using the function to evaluate each one.

## K-Nearest Neighbors Regression

K-Nearest Neighbors is a non-parameteric method that makes predictions for a new observation based on the K nearest observations based on a distance measure. For this implementation, the distance measure is the L2 norm or Euclidean distance (the Minkowski measure with p = 2). There is no actual learning done during the training phase of K-Nearest Neighbors because the work of calculating the nearest neighbors and making an estimation is done at testing time (this is an example of a "lazy learner"). K-Nearest Neighbors is extremely sensitive to the `n_neighbors` hyperparameter which we have set at 10 (up from the default 5) for this case. 

In [84]:
from sklearn.neighbors import KNeighborsRegressor

model = KNeighborsRegressor(n_neighbors = 10)
model

In [85]:
# Call the model function
knn_results = implement_model(model, train, train_targets, test, 
                              test_targets, model_name = 'KNN')
knn_results

['KNN', 19.3753323760111, 2.79544410737617, 23.656239736972875]

# Support Vector Machine

The support vector machine works by transforming the data to a new high-dimensional feature space using a kernel. This procedure, called the kernel trick, is designed to make the original data, which may not be linearly separable, linearly separable in the high-dimensional feature space. The support vector machine is highly sensitive to a number of hyperparameters: the `kernel`, `gamma`, `C` (the error term), and `epsilon`. For this implementation, we will use the default hyperparameters in Scikit-Learn which are:

* `kernel=rbf`: Gaussian Radial Basis Function kernel
* `gamma=auto`: Gamma is equal to $\frac{1}{\text{n_features}}$
* `C=1.0`: penalty parameter of the error term
* `epsilon=0.1`

In [86]:
from sklearn.svm import SVR

In [87]:
model = SVR()
model

SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='auto',
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [ ]:
svm_results = implement_model(model, train, train_targets, test, 
                              test_targets, model_name = 'svm')
svm_results

## Random Forest

The random forest is an ensemble method which makes one powerful estimator out of a number of simpler models. The random forest is an example of a bagging ensemble method in which all of the individual learners are trained independently and then predictions are made by averaging the predictions of the individuals. This allows for efficient training because the indidual decision tree regressors can be trained in parallel.

The random forest is fast to train, can be interpreted via the model feature importances, is very accurate, and has only one main hyperparameter to adjust, the number of trees in the forst. Other hyperparameters can be used to control the amount of overfitting by modifying the individuals in the forest. A random forest has less variance than a single decision tree which is very prone to overfitting. The "random" in the name comes from the fact that the model only trains on a subsample of the training examples (chosen with replacement called "bootstrapping") and only on a subset of the features. Both of these behaviors can be controlled through the model hyperparameters.

For this case, we will increase the number of trees (`n_estimators`) and leave the rest of the model hyperparameters at the default values.

In [88]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(n_estimators = 100, n_jobs = -1)
model

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [90]:
rf_results = implement_model(model, train, train_targets, test, 
                             test_targets, model_name='rf')
rf_results

['rf', 74.06617799873857, 0.23965796381071414, 15.798131022187153]

## ExtraTrees Regressor

The extra trees regressor is another bagging ensemble method. As with the random forest, the individual predictors are decision tree regressors and the ensemble makes a prediction by averaging the predictions of all the individuals. The difference between the extra trees model and a random forest is that the node splits are made on a random threshold in the extra trees ensemble. In other words, the values for splits of nodes are chosen randomly rather than by trying out all possible values as in a random forest. The "extra" means "extra random" because of this behavior. An additional difference is that the training observations are not sampled for extra trees (this behavior can be changed through the `bootstrap` hyperparameter in the function call). 

The extra trees model can be slightly faster than the random forest for training because the splits are chosen randomly instead of evaluating all splits for the optimal split. The idea behind extra trees is that the model will have lower variance than the random forest which can improve generalization performance on the test set. We will use 100 trees (`n_estimators`) and keep the other hyperparameters at the defaults. 

In [116]:
from sklearn.ensemble import ExtraTreesRegressor

model = ExtraTreesRegressor(n_estimators=100, n_jobs = -1)
model

ExtraTreesRegressor(bootstrap=False, criterion='mse', max_depth=None,
          max_features='auto', max_leaf_nodes=None,
          min_impurity_decrease=0.0, min_impurity_split=None,
          min_samples_leaf=1, min_samples_split=2,
          min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
          oob_score=False, random_state=None, verbose=0, warm_start=False)

In [117]:
et_results = implement_model(model, train, train_targets, test, 
                             test_targets, model_name = 'et')
et_results

['et', 8.696371039817677, 0.09951267198266578, 18.42033529685648]

## AdaBoost

The AdaBoost - standing for Adaptive Boosting - algorithm is another ensemble method. However, unlike the random forest and extra trees, it is a boosting ensemble and not a bagging ensemble. The difference is that the individual learners are not trained independently, but rather sequentially, with each learner learning from the mistakes of the previous. In AdaBoost, this takes the form of re-weighting the observations the previous learner got most wrong when training the next. AdaBoost trains on a random subset of the training examples, and the training examples with greater weight are more likely to be included in the training set. This makes the examples that are more difficult - missed more often - more likely to appear in the training set. Moreover, while bagging methods simply average the predictions of each individual, boosting methods weight the predictions of each individual based on it's error rate. Better learners are given exponentially more weight in the final prediction. 

Boosting methods can be built on any individual learner, but the most often choice is the decision tree. These decision trees are kept very small (sometimes decision trees with only one level are used which are called decision stumps) and on their own are weak learners. Weak learners perform better than random guessing, but are not very accurate. However, by adding a series of weak learners to the ensemble sequentially, the overall model is a strong classifier. Boosting is a general method that can be applied to any weak learner and AdaBoost was one of the first successful implementations. In later work, we will look at another boosting method called Gradient Boosting which is now considered more capable than AdaBoost. We will set the following hyperparameters:

* `n_estimators=1000`: number of decision trees. This is set high because the individual learners are weak
* `learning_rate=0.05`: the contribution of each learner to the ensemble. There is a tradeoff between `n_estimators` and the `learning_rate` with a lower learning rate complementing a higher number of weak learners.

The other hyperparameters will be set at the default values. The default base learner for AdaBoost is the `DecisionTreeRegressor`. AdaBoost cannot be trained in parallel which means we cannot set the `n_jobs` training argument.

In [122]:
from sklearn.ensemble import AdaBoostRegressor

model = AdaBoostRegressor(n_estimators = 1000, learning_rate = 0.05)
model

AdaBoostRegressor(base_estimator=None, learning_rate=0.05, loss='exponential',
         n_estimators=1000, random_state=None)

In [123]:
adaboost_results = implement_model(model, train, train_targets, test, 
                                    test_targets, model_name = 'adaboost')
adaboost_results

['adaboost', 236.52357400107576, 1.7747365153045394, 39.394952931553576]

# Implementing all Models

We can use the function we developed to test all of the machine learning models. This function will take in a building energy dataframe and return a dataframe of metrics for all the models developed here. The function will call the `implement_model` function once for each model.  

In [124]:
def evaluate_models(df):
    """Evaluate scikit-learn machine learning models
    on a building energy dataset.
    
    
    Parameters
    --------
    df : dataframe
        Building energy dataframe. Each row must have one observation
        and the the columns must contain the features. The dataframe
        needs to have an "elec_cons" column to be used as targets. 
    
    Return
    --------
    results : dataframe, shape = [n_models, 4]
        Modeling metrics. A dataframe with columns:
        model, train_time, test_time, mape. More models can be added
        to the function if required. 
        
    """
    try:
        # Preprocess the data for machine learning
        train, train_targets, test, test_targets = preprocess_data(df, test_days = 183, scale = True)
    except Exception as e:
        print('Error processing data: ', e)
        return
        
    # elasticnet
    model = ElasticNet(alpha = 1.0, l1_ratio=0.5)
    elasticnet_results = implement_model(model, train, train_targets, test, 
                                         test_targets, model_name = 'elasticnet')
    
    # knn
    model = KNeighborsRegressor()
    knn_results = implement_model(model, train, train_targets, test, 
                                  test_targets, model_name = 'knn')
    
    # svm
    model = SVR()
    svm_results = implement_model(model, train, train_targets, test, 
                                   test_targets, model_name = 'svm')
    
    # rf
    model = RandomForestRegressor(n_estimators = 100, n_jobs = -1)
    rf_results = implement_model(model, train, train_targets, test, 
                                  test_targets, model_name = 'rf')
    
    # et
    model = ExtraTreesRegressor(n_estimators=100, n_jobs = -1)
    et_results = implement_model(model, train, train_targets, test, 
                                  test_targets, model_name = 'et')
    
    # adaboost
    model = AdaBoostRegressor(n_estimators = 1000, learning_rate = 0.05, 
                              loss = 'exponential')
    adaboost_results = implement_model(model, train, train_targets, test, 
                                       test_targets, model_name = 'adaboost')
    
    # Put the results into a single array (stack the rows)
    results = np.vstack((elasticnet_results, knn_results, svm_results,
                         rf_results, et_results, adaboost_results))
    
    # Convert the results to a dataframe
    results = pd.DataFrame(results, columns = ['model', 'train_time', 'test_time', 'mape'])
    
    return results

In [ ]:
results = evaluate_models(df)

In [ ]:
results